In [ ]:
import geopandas as gpd
import rasterio
from rasterio.sample import sample_gen
import numpy as np
import pandas as pd

### 01 - Calculating individual school exposure

#### Sample climate hazard exposure values at school locations
Overlaying school locations with the climate hazard exposure rasters to determine each school's exposure to the respective hazards. For each hazard, a binary 0/1 column is added to the school location dataframe, indicating for each given school whether it is exposed to the respective hazard or not.

In [4]:
# Load school shapefile
schools = gpd.read_file("0001_BASE DATA FILES/01_Schools/00_GLOBAL FINAL/schools_final.shp")

In [ ]:
# Load coastal flooding raster and sample

coastal_flooding = "0001_BASE DATA FILES/02_Climate/03_Coastal Flooding/Exposure_CoastalFlooding.tif"


# Sample coastal flooding raster values at each school location and store as 'coastflood' column in school dataframe

with rasterio.open(coastal_flooding) as src:
    coords = [(x,y) for x, y in zip(schools.geometry.x, schools.geometry.y)]
    schools['coastflood'] = [val[0] if val[0] is not None else 0 for val in src.sample(coords)]


In [ ]:
# Load riverine flooding raster and sample

riverine_flooding = "0001_BASE DATA FILES/02_Climate/02_Riverine Flooding/Exposure_RiverineFlooding.tif"


# Sample riverine flooding raster values at each school location and store as 'rivflood' column in school dataframe

with rasterio.open(riverine_flooding) as src:
    coords = [(x,y) for x, y in zip(schools.geometry.x, schools.geometry.y)]
    schools['rivflood'] = [val[0] if val[0] is not None else 0 for val in src.sample(coords)]


In [ ]:
# Load water scarcity raster and sample

water_scarcity = "0001_BASE DATA FILES/02_Climate/01_Water Scarcity/Exposure_WaterScarcity.tif"


# Sample water scarcity raster values at each school location and store as 'watersc' column in school dataframe

with rasterio.open(water_scarcity) as src:
    coords = [(x,y) for x, y in zip(schools.geometry.x, schools.geometry.y)]
    schools['watersc'] = [val[0] if val[0] is not None else 0 for val in src.sample(coords)]
    

In [ ]:
# Load heatwaves raster and sample

heatwaves = "0001_BASE DATA FILES/02_Climate/05_Heatwaves/Exposure_Heatwaves9.tif"


# Sample heatwaves raster values at each school location and store as 'heatwvs' column in school dataframe

with rasterio.open(heatwaves) as src:
    coords = [(x,y) for x, y in zip(schools.geometry.x, schools.geometry.y)]
    schools['heatwvs'] = [val[0] if val[0] is not None else 0 for val in src.sample(coords)]
    

In [ ]:
# Load tropical cyclones lower threshold raster and sample

cyclones_lower = "0001_BASE DATA FILES/02_Climate/04_Tropical Cyclones/Exposure_Cyclones_LowTH.tif"


# Sample cyclones lower threshold raster values at each school location and store as 'cyclns_l' column in school dataframe

with rasterio.open(cyclones_lower) as src:
    coords = [(x,y) for x, y in zip(schools.geometry.x, schools.geometry.y)]
    schools['cyclns_l'] = [val[0] if val[0] is not None else 0 for val in src.sample(coords)]
    

In [ ]:
# Load tropical cyclones upper threshold raster and sample

cyclones_upper = "0001_BASE DATA FILES/02_Climate/04_Tropical Cyclones/Exposure_Cyclones_HighTH.tif"


# Sample cyclones upper threshold raster values at each school location and store as 'cyclns_u' column in school dataframe

with rasterio.open(cyclones_upper) as src:
    coords = [(x,y) for x, y in zip(schools.geometry.x, schools.geometry.y)]
    schools['cyclns_u'] = [val[0] if val[0] is not None else 0 for val in src.sample(coords)]
    

In [ ]:
# Load pm2.5 exposure lower threshold raster and sample

pm25_lower = "0001_BASE DATA FILES/02_Climate/06_Air Pollution/Exposure_PM25_LowTH.tif"


# Sample pm2.5 lower threshold raster values at each school location and store as 'pm25_l' column in school dataframe

with rasterio.open(pm25_lower) as src:
    coords = [(x,y) for x, y in zip(schools.geometry.x, schools.geometry.y)]
    schools['pm25_l'] = [val[0] if val[0] is not None else 0 for val in src.sample(coords)]
    

In [ ]:
# Load pm2.5 exposure upper threshold raster and sample

pm25_upper = "0001_BASE DATA FILES/02_Climate/06_Air Pollution/Exposure_PM25_HighTH.tif"


# Sample pm2.5 upper threshold raster values at each school location and store as 'pm25_u' column in school dataframe

with rasterio.open(pm25_upper) as src:
    coords = [(x,y) for x, y in zip(schools.geometry.x, schools.geometry.y)]
    schools['pm25_u'] = [val[0] if val[0] is not None else 0 for val in src.sample(coords)]
    

#### Dataframe cleanup
Cleaning up and processing the updated schools dataframe, ensuring it is ready for analysis

In [ ]:
# Checking the first few rows of the updated schools dataframe to verify exposure columns

schools.head(3)

,COUNTRY,GID_0,CONTINENT,name,amenity,grades,isced_leve,geometry,coastflood,rivflood,watersc,heatwvs,cyclns_l,cyclns_u,pm25_l,pm25_u
0,Nigeria,NGA,Africa,Ceta international school,school,None,None,POINT (5.60699 6.30471),0,1,0,1.0,-9999,-9999,1.0,1.0
1,Angola,AGO,Africa,Escola São José de Cluny,school,None,None,POINT (13.23774 -8.81384),0,0,1,1.0,-9999,-9999,1.0,0.0
2,Réunion,REU,Africa,École primaire de la Balance,school,None,None,POINT (55.27266 -21.03803),0,0,-9999,1.0,1,1,0.0,0.0


In [15]:
# Ensuring NA values are displayed as -9999 across all columns

schools = schools.fillna(-9999)

In [ ]:
# Checking data types

schools.dtypes

COUNTRY         object
GID_0           object
CONTINENT       object
name            object
amenity         object
grades          object
isced_leve      object
geometry      geometry
coastflood       int16
rivflood         int16
watersc          int16
heatwvs        float32
cyclns_l         int16
cyclns_u         int16
pm25_l         float32
pm25_u         float32
dtype: object

In [17]:
# Converting 'heatwaves', 'cyclones', 'pm25' columns to integers to ensure unity in dtypes

schools[['heatwvs', 'pm25_l', 'pm25_u']] = schools[['heatwvs', 'pm25_l', 'pm25_u']].astype('int16')

In [18]:
schools.head()

,COUNTRY,GID_0,CONTINENT,name,amenity,grades,isced_leve,geometry,coastflood,rivflood,watersc,heatwvs,cyclns_l,cyclns_u,pm25_l,pm25_u
0,Nigeria,NGA,Africa,Ceta international school,school,-9999,-9999,POINT (5.60699 6.30471),0,1,0,1,-9999,-9999,1,1
1,Angola,AGO,Africa,Escola São José de Cluny,school,-9999,-9999,POINT (13.23774 -8.81384),0,0,1,1,-9999,-9999,1,0
2,Réunion,REU,Africa,École primaire de la Balance,school,-9999,-9999,POINT (55.27266 -21.03803),0,0,-9999,1,1,1,0,0
3,Réunion,REU,Africa,École élémentaire et maternelle de Villèle,school,-9999,-9999,POINT (55.26029 -21.05377),0,0,-9999,1,1,1,0,0
4,Réunion,REU,Africa,Centre de formation professionnelle CENTHOR,school,-9999,-9999,POINT (55.26733 -21.04499),0,0,-9999,1,1,1,0,0


In [ ]:
# Saving the updated school dataframe as shapefile (spatial version)

schools.to_file("/0002_DATA PRODUCTS/schools_exposure.shp")

In [ ]:
# Saving the updated school dataframe as csv (non-spatial version, for faster processing)

schools.to_csv("0002_DATA PRODUCTS/schools_exposure.csv", index=False)

----------

## 02 - Calculating country-level exposure summaries

Calculating both the absolute and relative number of schools exposed per hazard for each country

In [ ]:
# Importing the updated school dataframe from csv

schools = pd.read_csv('/0002_DATA PRODUCTS/schools_exposure.csv')

In [ ]:
# Overview

schools

,COUNTRY,GID_0,CONTINENT,name,amenity,grades,isced_leve,geometry,coastflood,rivflood,watersc,heatwvs,cyclns_l,cyclns_u,pm25_l,pm25_u
0,Nigeria,NGA,Africa,Ceta international school,school,-9999,-9999,POINT (5.606988982531351 6.304707276898422),0,1,0,1,-9999,-9999,1,1
1,Angola,AGO,Africa,Escola São José de Cluny,school,-9999,-9999,POINT (13.237743273042197 -8.813844755871202),0,0,1,1,-9999,-9999,1,0
2,Réunion,REU,Africa,École primaire de la Balance,school,-9999,-9999,POINT (55.27265598373941 -21.038029983606826),0,0,-9999,1,1,1,0,0
3,Réunion,REU,Africa,École élémentaire et maternelle de Villèle,school,-9999,-9999,POINT (55.26028843192386 -21.053767166781903),0,0,-9999,1,1,1,0,0
4,Réunion,REU,Africa,Centre de formation professionnelle CENTHOR,school,-9999,-9999,POINT (55.267326150709515 -21.04498971259628),0,0,-9999,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380764,Jamaica,JAM,North America,RH404,school,-9999,-9999,POINT (-77.4982829 18.0158853),0,0,0,1,1,1,1,0
1380765,Jamaica,JAM,North America,Bathroom,school,-9999,-9999,POINT (-77.4982934 18.0159159),0,0,0,1,1,1,1,0
1380766,Cuba,CUB,North America,Escuela Primaria,school,-9999,-9999,POINT (-76.0826454 20.7706005),0,0,1,1,1,1,1,0
1380767,Cuba,CUB,North America,Escuela Primaria,school,-9999,-9999,POINT (-79.6219824 22.000882),0,0,1,1,1,1,0,0


In [ ]:
# Replacing -9999 / NAs for cyclones with 0 / not exposed, given the different nature of the cyclone exposure data

schools.loc[schools['cyclns_l'] == -9999, 'cyclns_l'] = 0
schools.loc[schools['cyclns_u'] == -9999, 'cyclns_u'] = 0

In [ ]:
# Replacing all other -9999 with NaN for cleaner handling in calculation

schools.replace(-9999, pd.NA, inplace=True)

In [ ]:
# Selecting hazard columns: all numeric except ISO3/country

hazard_columns = ['coastflood', 'rivflood', 'watersc', 'heatwvs', 'cyclns_l', 'cyclns_u', 'pm25_l', 'pm25_u']

In [ ]:
# Calculating exposure summaries for each hazard

summary_list = []

for hazard in hazard_columns:
    # Removing missing values
    temp = schools[['GID_0', hazard]].dropna()

    # Counting exposure levels
    counts = temp.groupby(['GID_0', hazard]).size().unstack(fill_value=0)
    
    # Renaming columns to reflect exposure levels & convert to strings
    counts.columns = [f"{hazard}_{int(col)}" for col in counts.columns]

    # Computing totals for proportion
    totals = counts.sum(axis=1)

    # Adding proportion columns (excluding exposure_0)
    for col in counts.columns:
        if col.endswith('_0'):
            continue
        pct_col = col + "_pct"
        counts[pct_col] = counts[col] / totals

    summary_list.append(counts)

In [ ]:
# Combining all summaries

countries_exposure = pd.concat(summary_list, axis=1).reset_index()


In [ ]:
# Joining back country names to newly created countries_exposure df, based on GID_0 (ISO3 code)

if 'COUNTRY' in schools.columns:
    countries = schools[['GID_0', 'COUNTRY']].drop_duplicates()
    countries_exposure = countries_exposure.merge(countries, on='GID_0', how='left')
    
    # Reordering columns for more userfriendliness: country, ISO3, then the rest
    cols = ['COUNTRY', 'GID_0'] + [col for col in countries_exposure.columns if col not in ['COUNTRY', 'GID_0']]
    countries_exposure = countries_exposure[cols]

In [ ]:
# Checking the updated countries_exposure df

countries_exposure

,COUNTRY,GID_0,coastflood_0,coastflood_1,coastflood_1_pct,rivflood_0,rivflood_1,rivflood_1_pct,watersc_0,watersc_1,...,cyclns_l_1_pct,cyclns_u_0,cyclns_u_1,cyclns_u_1_pct,pm25_l_0,pm25_l_1,pm25_l_1_pct,pm25_u_0,pm25_u_1,pm25_u_1_pct
0,Aruba,ABW,136.0,0.0,0.0,0.0,136.0,1.000000,136.0,0.0,...,1.0,136,0,0.0,80.0,56.0,0.411765,136.0,0.0,0.000000
1,Afghanistan,AFG,1713.0,0.0,0.0,109.0,1604.0,0.936369,251.0,1462.0,...,0.0,1713,0,0.0,3.0,1710.0,0.998249,804.0,909.0,0.530648
2,Angola,AGO,875.0,0.0,0.0,722.0,153.0,0.174857,642.0,233.0,...,0.0,875,0,0.0,1.0,874.0,0.998857,875.0,0.0,0.000000
3,Anguilla,AIA,8.0,0.0,0.0,8.0,0.0,0.000000,8.0,0.0,...,1.0,0,8,1.0,8.0,0.0,0.000000,8.0,0.0,0.000000
4,Åland,ALA,25.0,0.0,0.0,24.0,1.0,0.040000,25.0,0.0,...,0.0,25,0,0.0,25.0,0.0,0.000000,25.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,South Africa,ZAF,8553.0,0.0,0.0,7064.0,1489.0,0.174091,1274.0,7279.0,...,0.0,8557,0,0.0,2153.0,6404.0,0.748393,7265.0,1292.0,0.150987
240,Zambia,ZMB,5081.0,0.0,0.0,3898.0,1183.0,0.232828,4047.0,1034.0,...,0.0,5081,0,0.0,0.0,5081.0,1.000000,5081.0,0.0,0.000000
241,Northern Cyprus,ZNC,152.0,0.0,0.0,152.0,0.0,0.000000,0.0,152.0,...,0.0,152,0,0.0,0.0,152.0,1.000000,152.0,0.0,0.000000
242,Zimbabwe,ZWE,3012.0,0.0,0.0,1808.0,1204.0,0.399734,562.0,2450.0,...,0.0,3012,0,0.0,342.0,2670.0,0.886454,3012.0,0.0,0.000000


In [ ]:
# Save as csv to use in the next notebook for index calculation

countries_exposure.to_csv("/0002_DATA PRODUCTS/countries_exposure.csv", index=False)